### 여러 페이지의 이미지 데이터를 수집하기
키워드로 검색후, 검색결과를 순차적으로 조회해서 상세페이지의 이미지 가져오기

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import math
import os
import urllib
import urllib.request

In [2]:
print("="*70)
print("예제: 대한민국 구석구석 사이트의 여행지 정보 수집하기")
print("="*70)

query_txt = input("크롤링할 키워드는 입력:")
cnt = int(input("크롤링할 건색 건수:"))
item_per_page=10
page_cnt = math.ceil(cnt/item_per_page)

예제: 대한민국 구석구석 사이트의 여행지 정보 수집하기
크롤링할 키워드는 입력:정동진
크롤링할 건색 건수:10


In [5]:
f_dir = input("결과 폴더들을 저장할 디렉토리 입력 (C:\\LHR\\python\\notebook\\data\\):")
n = time.localtime()
s = '{}-{}-{}-{}-{}'.format(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min)
dir_name=f_dir+s+'-'+query_txt
os.makedirs(dir_name)

dir_name +='\\'

결과 폴더들을 저장할 디렉토리 입력 (C:\LHR\python\notebook\data\):C:\LHR\python\notebook\data\


In [6]:
#driver 생성 및 웹사이트 소환
s_time = time.time()

path="C:/LHR/python/datadown/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://korean.visitkorea.or.kr/")
time.sleep(2)

try:
    driver.find_element_by_xpath('//*[@id="chkForm01"]').click()
except:
    print('코로나 창이 없습니다.')

#검색실행
element = driver.find_element_by_id("inp_search")
element.send_keys(query_txt)
element.send_keys("\n")

driver.maximize_window()

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

print("="*80)
print('크롤링할 키워드:',query_txt)
print('크롤링할 총 페이지 번호:',page_cnt)
print("="*80)

크롤링할 키워드: 정동진
크롤링할 총 페이지 번호: 1


In [7]:

no =1 #검색건수 카운터 (1~cnt)
catalog = {} #상세페이지 title별 이미지리스트 dictionary: key:title, value:img_src2리스트

for x in range(1,page_cnt+1):
    print('{}페이지의 데이터를 가져옵니다.============='.format(x))
    
    for y in range(1,item_per_page+1):
        #요청한 검색건수에 도달하면 for loop나오기
        if no > cnt:
            break
            
        #검색결과들의 상세페이지로 이동    
        try:
            driver.find_element_by_xpath('//*[@id="listBody"]/ul/li[{}]/div[2]/div[1]/a'.format(y)).click()
        except:
            continue
        time.sleep(2)
        
        #상세페이지 소스코드 받아서 soup객체 생성 
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        img_src1=[] # 이미지url 리스트
        
        #상세페이지 유형별로 이미지 가져오기
        try:
            try:
                #상세페이지 제목받아오기 (폴더이름으로 사용예정)
                title = soup.find('div', id='contents').find('h2').get_text()

                #페이지에서 tab별 이미지url 받아오기
                num_thumbs=4
                for z in range(1,num_thumbs+1):
                    driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[6]/div[1]/div[1]/div/ul/li[%s]/a'%z).click()
                    time.sleep(1)

                    html = driver.page_source
                    soup = BeautifulSoup(html,'html.parser')

                    tab = soup.find('div',"cos_cont active")
                    tab2 = tab.find('div','info_area pc').find_all('a')

                    for i in tab2:
                        img_src = i['style'].split("(")[1].split(")")[0]
                        img_src1.append(img_src) #리스트에 저장

                catalog[title]=img_src1  
                print(no,') 유형1:',title)
            except:
                try:
                    #상세페이지 제목받아오기 (폴더이름으로 사용예정)
                    title = soup.find('div','tit_cont titleType1').find('h2', id='topTitle').get_text()

                    #상세페이지 이미지 받아오기
                    img_src = soup.find('div','box_txtPhoto').find_all('img')
                    for i in img_src:
                        img_src=i['src']
                        img_src1.append(img_src)

                    catalog[title]=img_src1
                    print(no,') 유형2:',title)
                except:
                    print(no,') 유형3: 이미지를 가져올 수 없습니다.')
                    
        except:
            print('no',no)
            no +=1
            driver.back()
            continue
            
        print('no',no)
        no +=1
        driver.back()
        time.sleep(2)
        #여기까지 item별 loop


    x += 1
    print('x',x)             
    if(x % 5 == 1):
        driver.find_element_by_link_text("다음").click()
    else:
        driver.find_element_by_link_text("{}".format(x)).click()
        
    time.sleep(2)  
    #여기까지 page별 loop                  

1페이지의 데이터를 가져옵니다.=============
1 ) 유형1: 4코스새해, 첫 일출은 정동진에서
no 1
2 ) 유형2: [강릉] 정동진 근처 가성비 좋은 숙소, 카리브모텔
no 2
3 ) 유형2: 정동진 모래시계 공원(2016년 선정 열린관광지 )
no 3
4 ) 유형2: 해맞이 기차여행, 정동진
no 4
5 ) 유형3: 이미지를 가져올 수 없습니다.
no 5
6 ) 유형3: 이미지를 가져올 수 없습니다.
no 6
7 ) 유형2: 경포대냐 정동진이냐, 그것이 문제로다! 
no 7
8 ) 유형2: 일출여행 준비 완료 정동진 무박 2일
no 8
9 ) 유형2: 정동진, 새로운 주인공은 나야 나! 정동심곡바다부채길
no 9
x 2


In [8]:
f_dir_list=[] #폴더 리스트
title_list = list(catalog.keys())
img_list = list(catalog.values())

print(title_list)
print(img_list)

#title별 폴더만들기
for t in title_list:           
    now = time.localtime()
    s = '{}-{}-{}-{}-{}' .format(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min)
    f_dir_name = dir_name+s+'-'+t
    os.makedirs(f_dir_name)
    #폴더 리스트에 추가
    f_dir_list.append(f_dir_name)
 
#폴더별로 이미지 저장  
for i in range(0,len(img_list)):
    file_no=0 #이미지 순번   
    
    #이미지를 저장하려는 위치로 디렉토리를 변경
    os.chdir(f_dir_list[i])
    print("폴더: "+f_dir_list[i]+"에 저장 시작:")    
    for j in range(0,len(img_list[i])):
        try:
            urllib.request.urlretrieve(img_list[i][j],str(file_no)+'.jpg')
        except:
            continue
        file_no+=1
        time.sleep(0.5)
        print("%s번째 이미지 저장중입니다."%file_no)

['4코스새해, 첫 일출은 정동진에서', '[강릉] 정동진 근처 가성비 좋은 숙소, 카리브모텔', '정동진 모래시계 공원(2016년 선정 열린관광지 )', '해맞이 기차여행, 정동진', '경포대냐 정동진이냐, 그것이 문제로다! ', '일출여행 준비 완료 정동진 무박 2일', '정동진, 새로운 주인공은 나야 나! 정동심곡바다부채길']
[['https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id=78681fbe-bb50-401a-880d-e4b61b58a178', 'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id=2d227fcc-4ff4-4e2c-925f-df37a4ad7469', 'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id=866afd3f-9f3c-4203-ab5a-8ac1ea738ed0', 'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id=0e48b7f7-4059-4f65-948a-c4b6d274bbd3', 'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id=ecc3281a-19eb-410c-932c-8e37298c2538', 'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id=7ff7d5d8-e353-40be-82d1-8a1830ace3b6', 'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id=5039fd4c-560f-4990-b6dd-0f1f4f9114c7', 'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id=fb3b9b04-1b2d-4004-914f-24dd361c2291', 'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id=41267d26-2c03-40ce-94ec-9efc612c0baf', '

1번째 이미지 저장중입니다.
2번째 이미지 저장중입니다.
3번째 이미지 저장중입니다.
4번째 이미지 저장중입니다.
5번째 이미지 저장중입니다.
6번째 이미지 저장중입니다.
7번째 이미지 저장중입니다.
8번째 이미지 저장중입니다.
9번째 이미지 저장중입니다.
10번째 이미지 저장중입니다.
11번째 이미지 저장중입니다.
12번째 이미지 저장중입니다.
폴더: C:\LHR\python\notebook\data\2021-4-16-16-11-정동진\2021-4-16-16-13-[강릉] 정동진 근처 가성비 좋은 숙소, 카리브모텔에 저장 시작:
1번째 이미지 저장중입니다.
2번째 이미지 저장중입니다.
3번째 이미지 저장중입니다.
4번째 이미지 저장중입니다.
5번째 이미지 저장중입니다.
6번째 이미지 저장중입니다.
7번째 이미지 저장중입니다.
8번째 이미지 저장중입니다.
9번째 이미지 저장중입니다.
10번째 이미지 저장중입니다.
11번째 이미지 저장중입니다.
12번째 이미지 저장중입니다.
13번째 이미지 저장중입니다.
14번째 이미지 저장중입니다.
15번째 이미지 저장중입니다.
폴더: C:\LHR\python\notebook\data\2021-4-16-16-11-정동진\2021-4-16-16-13-정동진 모래시계 공원(2016년 선정 열린관광지 )에 저장 시작:
1번째 이미지 저장중입니다.
2번째 이미지 저장중입니다.
3번째 이미지 저장중입니다.
4번째 이미지 저장중입니다.
5번째 이미지 저장중입니다.
6번째 이미지 저장중입니다.
7번째 이미지 저장중입니다.
8번째 이미지 저장중입니다.
9번째 이미지 저장중입니다.
10번째 이미지 저장중입니다.
11번째 이미지 저장중입니다.
12번째 이미지 저장중입니다.
13번째 이미지 저장중입니다.
14번째 이미지 저장중입니다.
15번째 이미지 저장중입니다.
16번째 이미지 저장중입니다.
17번째 이미지 저장중입니다.
18번째 이미지 저장중입니다.
19번째 이미지 저장중입니다.
20번째 이미지 저장중입니다.
21번째 이미지 저장중입니다.
2

In [ ]:
e_time = time.time()
d_time = e_time - s_time

print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(d_time,1))
total=0
for i in img_list:
    total += len(i)
print("총 저장 건수는 %s 건 입니다 " %(total))
print("파일이 저장된 폴더들의 경로는 입니다")
for x in f_dir_list:
    print(x)
print("=" *80)
 
driver.close( )